In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score

Functions to help find accuracy later:

In [2]:
def get_top_k_predictions(model,X_test,k):
    
# get probabilities instead of predicted labels, since we want to collect top 3
    probs = model.predict_proba(X_test)
# get top k probabilities
    best_n = np.argsort(probs, axis=1)[:,-k:]
    
# get category
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    
# reverse category
    preds=[ item[::-1] for item in preds]
        
    return preds


In [3]:
def get_accuracy(predictions,actual):
    #function is given top 3 predictions and the actual genres of movies
    lengthdf = len(actual)
    lengthk = len(predictions)
    total_right = 0
    #check if any of the predicted genres match any of the actual genres
    for i in range(lengthdf):
        for j in range(3):
            if(actual.iloc[i,j] == predictions[i][0] or actual.iloc[i,j] == predictions[i][1] or actual.iloc[i,j] == predictions[i][2]):
                total_right += 1
    accuracy = total_right/(actual['genre'].sum())
    return accuracy


In [4]:
print('Starting to read Bag of Words')
movies = pd.read_csv('PorterBagOfWords.csv') #read bag of words this one only contains one column of genres

print('Finished reading Bag of Words')

extragenres= pd.read_csv('otherGenres.csv') #this file contains the other column of genres we are just adding them together 
movies['genre2'] = extragenres['genre2'] 
movies['genre3'] = extragenres['genre3']
#resetting index and creating training and test data sets
movies = movies.reset_index(drop =True)
training_data,testing_data = train_test_split(movies,random_state=0)
#this is just a dataframe with the genres of the testing dataset
all_genres = pd.DataFrame()
all_genres[['genre','genre2','genre3']] = testing_data[['genre','genre2','genre3']]
all_genres = all_genres.reset_index(drop=True)
#separating into bag of words and genre labels
Y_train = training_data['genre'].values
Y_test = testing_data['genre'].values
X_train= training_data.drop(labels= ['genre','genre2','genre3'], axis=1)
X_test= testing_data.drop(labels= ['genre','genre2','genre3'], axis=1)

#just a list of categories
categories = ['Action', 'Drama', 'Thriller', 'Crime', 'Adventure', 'Comedy', 'Sci-Fi', 'Romance','Fantasy','Horror','War','Mystery','Western','History','Family','Animation','Sport','Film-Noir']


Starting to read Bag of Words


FileNotFoundError: ignored

In [5]:
#creating Multinomial and Complement models
MNB = MultinomialNB()
CNB = ComplementNB()
multiModel = MNB.fit(X_train, Y_train)
compModel = CNB.fit(X_train, Y_train)
#getting 1 genre predictions
compYPreds = CNB.predict(X_test)
multiYPreds = MNB.predict(X_test)
#getting top 3 probabailities
top3Comp = get_top_k_predictions(multiModel,X_test,3)
top3Multi = get_top_k_predictions(compModel,X_test,3)
#calculating accuracy with top 3 probs
acc1 = get_accuracy(top3Comp,all_genres)
acc2 = get_accuracy(top3Multi, all_genres)

print('Accuracy for top 3 Complement Naive Bayes predictions: ',acc1)
print('Accuracy for top 3 Multinomial Naive Bayes predictions:',acc2)
print('\nAccuracy for single prediction Complement Naive Bayes: ', metrics.accuracy_score(Y_test,compYPreds))
print('Accuracy for single prediction Multinomial Naive Bayes: ', metrics.accuracy_score(Y_test,multiYPreds))

print('\n Complement NB scores:')
print('Precision score: ', precision_score(Y_test,compYPreds,average="micro"),'\nRecall score:', recall_score(Y_test,compYPreds,average="micro"),'\nF1 score:', f1_score(Y_test,compYPreds,average="micro"))
print('\n Multinomial NB scores:')
print('Precision score: ', precision_score(Y_test,multiYPreds,average="micro"),'\nRecall score:', recall_score(Y_test,multiYPreds,average="micro"),'\nF1 score:', f1_score(Y_test,multiYPreds,average="micro"))

NameError: ignored